### Introduction to NLP Fundamentals in TensorFlow

    NLP has the goal of deriving information out of natural language (could be sequences text or speech)

    Another common term for NLP problems is sequence to sequence problems (seq2seq)

In [2]:
# Check for GPU
!nvidia-smi -L

GPU 0: NVIDIA GeForce GTX 1650 (UUID: GPU-890aadbc-dc9a-cfdd-ce0d-dd3bdfa0ac28)


In [3]:
 ## GET helper functions
import sys
sys.path.append(r"C:\Users/wjub/01workspace/TensorFlow Developer Certificate ZTM")
import winsound
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import helper_functions

In [4]:
# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

# Get a text dataset

    The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as disaste or not disasterrs)

    See the original source here : https://www.kaggle.com/competitions/nlp-getting-started/data

In [5]:
helper_functions.download_file("https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip","nlp_getting_started.zip")

In [6]:
helper_functions.unzip_data("nlp_getting_started.zip")

## Visualizing a text data 

        To visualize our text samples, we first have to read them in, one way to do so would be to use Python :

        So another way to do this is to use pandas.

In [7]:
import pandas as pd
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")
train_df.head()
# train_df["text"][0]

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [8]:
# Shuffle training dataframe
# frac : nb of % to shuffle 1=100%
train_df_shuffled = train_df.sample(frac=1, random_state = 42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [9]:
# What does the test dataframe look like ?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [10]:
# How many examples of each class ?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [11]:
# How many total sample ?
len(train_df), len(test_df)

(7613, 3263)

In [12]:
#  Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5) # create random indexes not higher than the total number of  samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
    _, text, target = row
    print(f"Target:{target}", "(real diaster)" if target > 0 else "(not real diaster)")
    print(f"Text:\n{text}\n")
    print("---\n")

Target:1 (real diaster)
Text:
IG: http://t.co/2WBiVKzJIP 'It's hailing again! #abstorm #yyc #hail #hailstorm #haildamage #yycweather #calgary #captureyyc #alberta #stoÛ_

---

Target:0 (not real diaster)
Text:
Britney Spears &gt; Vegas!! I missed you so much! ?? CanÛªt wait for the show tonight ?? #PieceOfMe 

?? Catastrophe ?? http://t.co/4mJyW7p7Cf

---

Target:0 (not real diaster)
Text:
Panic! at the Disco is the best song by this is gospel (acoustic) http://t.co/VCq2icptKI

---

Target:1 (real diaster)
Text:
Something Catastrophic Is Coming: Should We Tune Out? http://t.co/a8jZ5A26wi

---

Target:1 (real diaster)
Text:
#TheaterShooting DEFENDANT/MASS MURDERER CHOSE NOT 2 TESTIFY IN FINAL PHASE 3 so he won't therefore B subject 2 cross-exam or jury questions

---



### Split data into training and validation sets

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
# turn dataframe values to dataframe columns
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                    train_df_shuffled["target"].to_numpy(),
                                                    test_size = 0.1, # use 10% of traianing data for validations
                                                    random_state = 42)

In [15]:
# Check the lengths
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

(6851, 762, 6851, 762)

In [16]:
# check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers

    When dealing with a text problem, on of the first things you'll have to do before you can build a model is to convert your text to numbers.

    There are a few ways to do this, namely:
     * Tokenization - direct mapping of token (a token could be a word or a caracter) to number
     * Embedding - create a matric of feature vector for each token (the size of the feature vector can be defined and this embedding can be learned)

# Text vectorization (tokenization)

In [17]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [18]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization parameters
text_vectorization = TextVectorization(max_tokens=50000, # how many words in the vocabulary(utomatically add <OOV>)
                                       standardize="lower_and_strip_punctuation",
                                       split="whitespace",
                                       ngrams=None, # Create groups of n-words?
                                       output_mode="int", # How to map tokens to numbers
                                       output_sequence_length=None, # how longdo you want your sequences to be?
                                       pad_to_max_tokens=True)

In [19]:
train_sentences[0].split()

['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet']

In [20]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])) # nb of words
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences)) # average for 1 sentence

15

In [21]:
# Setup text vectorization variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be 

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                   output_mode = "int",
                                   output_sequence_length= max_length)

In [22]:
# Fit the text vectorizer  to the training text 
# that convert our words data to numeric format 
text_vectorizer.adapt(train_sentences)

In [23]:
# Create a sample sentences and tokenize it 
sample_sentence = "There's a flood in my street !"
text_vectorizer([sample_sentence])


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]], dtype=int64)>

In [24]:
# Choose a random sentence from the training dataset and tokenize it 
random_sentence = random.choice(train_sentences)
print(f"Origina text:\n{random_sentence}\
\n\n Vectorized version:")
text_vectorizer([random_sentence])

Origina text:
The sign-up is open for the FALLING FOR DANGER release day blast and review tour. Sign-up here:... http://t.co/hbdo22nqPZ

 Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   2, 4622,    9, 1497,   10,    2, 1140,   10,  424, 2008,  101,
         557,    7, 1608, 1942]], dtype=int64)>

In [25]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # get all of the unique words in our training data
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] # get the least common words
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


## Creating an Embedding using an Embedding Layer

    To make our embedding, we re going to use TensorFlow's embedding layer : https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding
    The parameters we xare most about for our embedding layer:
    *`input_dim` = the size of our vocabulary
    *`output_dim` = the size of the output embedding vector, for example, a value of 100 could mean each token gets represented by a vector 100 long
    * `input_length` = length of the sequences being passed to the embedding layer

In [26]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128,
                             embeddings_initializer="uniform",
                             input_length=max_length)
embedding

In [27]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
\n\n Embedded version :")
# Embed the random sentence (turn it into danse victors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))

#sample_embed = tf.reshape(sample_embed, (1, 15, 128))
sample_embed

Original text:
devastated by today's allegations.

 Embedded version :


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.04919444, -0.02019825,  0.04034439, ..., -0.02313584,
          0.00683199, -0.01828556],
        [ 0.03810577,  0.01534942,  0.0127439 , ..., -0.04144097,
         -0.03877351, -0.03841438],
        [-0.03479196,  0.00683158,  0.01368764, ...,  0.01348775,
          0.01529488, -0.02029485],
        ...,
        [-0.01155154,  0.04900042,  0.04742882, ...,  0.00459126,
         -0.0087118 , -0.01219957],
        [-0.01155154,  0.04900042,  0.04742882, ...,  0.00459126,
         -0.0087118 , -0.01219957],
        [-0.01155154,  0.04900042,  0.04742882, ...,  0.00459126,
         -0.0087118 , -0.01219957]]], dtype=float32)>

In [28]:
# Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.04919444, -0.02019825,  0.04034439,  0.01323748, -0.01951959,
        -0.01728883, -0.03315293, -0.03535081,  0.01076172,  0.03336581,
         0.02450315, -0.02970837, -0.03891735,  0.04461154,  0.02963319,
         0.03082962, -0.00767208, -0.00109283,  0.02091951,  0.01021271,
         0.03978187, -0.03905947, -0.0071187 , -0.01246103, -0.04108756,
         0.01877992,  0.00139893,  0.01396311,  0.00583767,  0.0112428 ,
        -0.040424  ,  0.04356946,  0.00582137, -0.00665487, -0.00720727,
         0.01622306,  0.03493842, -0.01069515,  0.01102204,  0.02838646,
         0.00329549, -0.00431855, -0.02085346,  0.03951338,  0.04447876,
        -0.03628203, -0.00256868, -0.04309894,  0.00794926, -0.00996629,
         0.02391498,  0.03797312,  0.04718516, -0.00985303, -0.01309504,
         0.03020691, -0.01378341, -0.00031288, -0.02669017,  0.0389791 ,
        -0.00763347, -0.02038559,  0.01230538, -0.01452059,  0.03688279,
  

## Modelling a text dateset (running a series of experimente)

    Now we ve a got way to turn our text sequences into numbers,
    it's time to start building a series of modelling experiments.

    We'll start with a baseline and moce on from there.

* Model 0: Naive Bayes (baseline) with sklearn : https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html
* Model 1: Feed-forward neural network (danse model)
* Model 2 : LSTM model (RNN)
* model 3 : GRU model (RNN)
* model 4 : Bidirectional-LSTM model (RNN)
* Model 5 : 1D Convolutional Neural Network (CNN)
* Model 6 : TensorFlow Hub Pretrained Feature Extractor (using transfer learning for NLP)
* Model 7 : Same as model 6 with 10% of training data

How are we going to approch all of these ?

Use the standard steps in modelling with tensorflow:

* Create a model 
* Build a model
* Fit a model
* Evaluate model

### Model 0 : Getting a baseline

As with all machine learning modelling experiments, it's important to create a baseline model so you're got a benchmark for future experiments to build upon.

To create our baseline, we'll use Sklearn's Multinomial Naive Bayers using the TF-IDF formula to convert our words to numbers

>** note : ** it's common practice to use non-Deep Learning Algorithms as a baseline because of their speed and then later using Deep Learning to to see if you can imporove upon them.

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline ([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text clf : juste classification
])

# Fit the pipleine to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [30]:
# Evaluate the baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achives an accuracy of : {baseline_score}")

Our baseline model achives an accuracy of : 0.7926509186351706


In [31]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:10]
# for i in range(0,10,1):
#     print(f" {val_sentences[i]} \n prediction : {baseline_preds[i]}")
    

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0], dtype=int64)

### Creating an evaluation function for our model experiments

We could evaluate all of our model's predictions with different metrics every time, however this will be cumbersome and could easily be fixed with function

Let's create one to compare our model's predictions with the truth labels using the following metrics:
* Accuracy.
* Precision.
* Recall
* F1-score

For a deep overview of many different evaluation methods, see the sklearn

### 

In [32]:
# Function to evaluate : accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
# rajouter
def calculate_results(y_true, y_pred):
    """
    Calculates model accuracy, precision, recall and f1 score of a binary classification model.
    """
    # Calculate model accuracy
    model_accuracy = accuracy_score(y_true, y_pred) * 100
    # Calculate model precision, recall and f1-score using "weighted" average
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
    model_results = {"accuracy": model_accuracy,
                    "precision": model_precision,
                    "recall": model_recall,
                    "f1": model_f1}
    return model_results

In [33]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

# Model 1: A simple dense model

In [34]:
# Create a tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create a directory to save TensorBoard logs
SAVE_DIR = "model_logs"

In [35]:
# Reshape data
train_sentences = train_sentences.reshape(-1, 1)
train_labels = train_labels.reshape(-1, 1)
val_labels = val_labels.reshape(-1, 1)
val_sentences = val_sentences.reshape(-1, 1)
train_sentences.shape, train_labels.shape, val_labels.shape, val_sentences.shape

((6851, 1), (6851, 1), (762, 1), (762, 1))

In [36]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1), dtype=tf.string) # inputs are 1-dimensional string
x = text_vectorizer(inputs)# turn the input text into numbers
x = embedding(x) # transform the input numbers in embedding
x = layers.GlobalAveragePooling1D()(x) # Condense the feature vector for each token to on vector
outputs = layers.Dense(1, activation="sigmoid")(x) # Create the output layer, want binary outputs
model_1 = tf.keras.Model(inputs,outputs, name="Model_1_dense")

In [37]:
model_1.summary()    

Model: "Model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [38]:
model_1.compile(loss="binary_crossentropy",
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

In [39]:
history_model_1 = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                        experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20230109-201946
Epoch 1/5
215/215 [==============================] - 4s 7ms/step - loss: 0.6144 - accuracy: 0.6869 - val_loss: 0.5388 - val_accuracy: 0.7598
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.4431 - accuracy: 0.8168 - val_loss: 0.4758 - val_accuracy: 0.7835
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.3478 - accuracy: 0.8606 - val_loss: 0.4594 - val_accuracy: 0.7874
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2844 - accuracy: 0.8913 - val_loss: 0.4666 - val_accuracy: 0.7848
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2382 - accuracy: 0.9115 - val_loss: 0.4791 - val_accuracy: 0.7874


In [40]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4791 - accuracy: 0.7874


[0.47906991839408875, 0.787401556968689]

In [41]:
# Make some predictions
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape, model_1_pred_probs[0]

24/24 [==============================] - 0s 2ms/step


((762, 1), array([0.34930825], dtype=float32))

In [42]:
# Look at a single prediction
model_1_pred_probs[:10]

array([[0.34930825],
       [0.8164375 ],
       [0.9974873 ],
       [0.11807142],
       [0.08339188],
       [0.9495321 ],
       [0.90337974],
       [0.993796  ],
       [0.9686206 ],
       [0.2775715 ]], dtype=float32)

In [43]:
model_1_pred_probs.shape

(762, 1)

In [44]:
# Convert model prediction probabilities to label format

model_1_pred_probs = tf.squeeze(tf.round(model_1_pred_probs))
model_1_pred_probs[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [45]:
# Calculate our model_1 results
model_1_results = calculate_results(y_true=val_labels,
                                   y_pred=model_1_pred_probs)
model_1_results

{'accuracy': 78.74015748031496,
 'precision': 0.7927656871284042,
 'recall': 0.7874015748031497,
 'f1': 0.7842639469124086}

In [46]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [47]:
# compare witch best between our models
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values())) 

array([False, False, False, False])

## Visualizing learned embeddings


In [48]:
# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [49]:
max_vocab_length

10000

In [50]:
# Model 1 summary
model_1.summary()

Model: "Model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [52]:
# Get the weight matrix of embedding layer
# (thse are the numerical representations of each token in our training data, which have been learned for -5 epochs)
embed_weights = model_1.get_layer("embedding").get_weights()
print(embed_weights[0].shape) # same size as vocab size and embedding_dim (output_dim of our embedding layer)

(10000, 128)


In [53]:
embed_weights

[array([[ 0.0001717 ,  0.04524684,  0.04131022, ..., -0.00893129,
         -0.02286736,  0.00077061],
        [ 0.01970842, -0.00328306, -0.05034212, ..., -0.00048513,
         -0.04637272,  0.03199514],
        [ 0.05832125, -0.01618775, -0.06116477, ..., -0.02886888,
         -0.02301238,  0.02017602],
        ...,
        [-0.00945799,  0.01379514, -0.04897749, ..., -0.02623316,
          0.02737803, -0.03187831],
        [ 0.02039738,  0.0054359 , -0.0118881 , ..., -0.06560867,
         -0.01992019,  0.07448313],
        [ 0.09393812, -0.05755303, -0.09041245, ..., -0.08414539,
         -0.02578118,  0.05907102]], dtype=float32)]

Now we've got the embedding matric our model has learned to represent our tokens, let's see how we can visualize it.

To do so, TensorFlow has a handly tool called projector: https://projector.tensorflow.org/

And TensorFlow also has an incredible guide on word embeddings themselves
https://www.tensorflow.org/text/guide/word_embeddings

In [54]:
# # Create embedding files (we got this from TensorFlow's word embedding documentation)
# import io
# out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
# out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

# for index, word in enumerate(words_in_vocab):
#   if index == 0:
#     continue  # skip 0, it's padding.
#   vec = embed_weights[0][index]
#   out_v.write('\t'.join([str(x) for x in vec]) + "\n")
#   out_m.write(word + "\n")
# out_v.close()
# out_m.close()

In [55]:
# try:
#   from google.colab import files
#   files.download('vectors.tsv')
#   files.download('metadata.tsv')
# except Exception:
#   pass

Downloading the files above can visualize them usign http://projector.tensorflow.org/ and clicking the "load" button on the left hand
*Ressources : if you'd like to know more about embedding, I'd encourage you to check out :
- Jay Alammer's vusualized word2vec post : https://jalammar.github.io/illustrated-word2vec/
- TensorFlow's Word Embeddings guide: https://www.tensorflow.org/tutorials/text/word_embeddings

## Recurrent Neural Networks (RNN's)

RNN's are useful for sequence data.

The premise of a recurrent neural network is to use the representation of a previous input to aid the representation of a later
* Ressources :> 
If you want an overview of the internale of a recurrent neural network, see the following : 
    - MIT's sequence modelling lecture https://www.youtube.com/watch?v=QvkQ1B3FBqA
    - Chris Olah's intro to LSTMs : https://colah.github.io/posts/2015-08-Understanding-LSTMs/
    - Andrej Karphathy's the unreasonable effectiveness of recurrent neural network : http://karpathy.github.io/2015/05/21/rnn-effectiveness/